In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Documents/GitHub/sqlalchemy-challenge/hawaii.sqlite")

In [6]:
 # reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurements = Base.classes.measurement
Stations = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# It startes from the last data point in the database. 
# Then, it calculate the date one year from the last date in data set.
previous_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores from the data.
results = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= previous_year).all()

# Save query results as a Pandas DF and set the index to the date column.
df = pd.DataFrame(results, columns=['date', 'precipitation'])
df.set_index(df['date'], inplace=True)

# Sort the df by date
df = df.sort_values("date")

# Use Pandas Plotting w/ Matplotlib to plot the data for analysis
df.plot(rot=90)
plt.xlabel("Date")
plt.ylabel("Inches") 


In [ ]:
# Use Pandas and calculate the summary statistics 
df.describe()

In [ ]:
 # Create query to show to sow the number of stations available in this dataset
session.query(func.count(Stations.station)).all()

In [ ]:
# What are the most active stations, listeed and counted in desecnding orders?
session.query(Measurements.station, func.count(Measurements.station)).\
    group_by(Measurements.station).order_by(func.count(Measurements.station).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, 
# highest, and average temperature.
session.query(func.min(Measurements.tobs), func.max(Measurements.tobs), func.avg(Measurements.tobs)).\
    filter(Measurements.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temp observations.
# Query the last 12 months of temp observation data for this station. 
# Plot the results as a histogram.

import datetime as dt
from pandas.plotting import table
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

results = session.query(Measurements.tobs).\
    filter(Measurements.station == 'USC00519281').\
    filter(Measurements.date >= prev_year).all()
df = pd.DataFrame(results, columns=['tobs'])
df.plot.hist(bins=12)
plt.tight_layout()
plt.xlabel("Temperature")